In [1]:
!pip install transformers sentence-transformers faiss-cpu numpy
!pip install langchain
!pip install -U langchain-community
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 5.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 60.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take int

In [2]:
import os
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader


In [3]:

# Load Sentence-BERT model from Hugging Face (free version)
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight model for sentence embeddings



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [41]:

# Function to load and chunk text from a file
def chunk_text_from_file(file_path, chunk_size=500, chunk_overlap=50):
    loader = TextLoader(file_path)
    document = loader.load()
    
    # Split the document into chunks using RecursiveCharacterTextSplitter
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_documents(document)
    return chunks


In [42]:

# Function to generate embeddings for the text chunks
def generate_embeddings(text_chunks):
    embeddings = model.encode([chunk.page_content for chunk in text_chunks], convert_to_numpy=True)
    return embeddings


In [43]:

# Create FAISS index for storing the embeddings
def create_faiss_index(embeddings, dim=384):  # Dimension of 'all-MiniLM-L6-v2' is 384
    index = faiss.IndexFlatL2(dim)  # L2 distance metric for vector similarity
    index.add(embeddings)  # Add embeddings to the index
    return index


In [44]:
pinecone_chunks = []
# Store embeddings and paper IDs in FAISS index
def store_paper_embeddings_in_faiss(folder_path):
    all_embeddings = []
    paper_ids = []
    
    for file_number, file_name in enumerate(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.isfile(file_path) and file_name.endswith('.txt'):
            paper_id = "paper_" + str(file_number+2200)
            # paper_id = os.path.basename(file_path)
            chunks = chunk_text_from_file(file_path)
            embeddings = generate_embeddings(chunks)
            for i, value in enumerate(chunks):
                temp_dict = {}
                temp_dict["id"] = paper_id + "#chunk_" + str(i)
                temp_dict["values"] = embeddings[i]                
                temp_dict["metadata"] = {"paper_id": paper_id, "chunk_id": "chunk_" + str(i), "chunk": chunks[i].page_content}
                pinecone_chunks.append(temp_dict)

                # temp_dict["paper_id"] = paper_id
                # temp_dict["chunk_id"] = "chunk_" + str(i)
                # temp_dict["chunk"] = chunks[i].page_content
            all_embeddings.extend(embeddings)
            paper_ids.extend([paper_id] * len(embeddings))

    # Convert list to numpy array for FAISS
    embeddings_array = np.array(all_embeddings).astype('float32')
    # Create and store embeddings in FAISS index
    faiss_index = create_faiss_index(embeddings_array)
    
    # Save FAISS index to file (optional)
    faiss.write_index(faiss_index, "faiss_index_file.index")
    
    return faiss_index, paper_ids


In [45]:

# Query the index to find the closest matching research papers
def query_faiss_index(query_text, faiss_index, paper_ids, top_k=10):
    query_embedding = model.encode([query_text], convert_to_numpy=True)
    
    # Search the FAISS index
    D, I = faiss_index.search(query_embedding, top_k)
    # print(D[0], I[0])
    # Return the paper IDs of the closest matches
    results = [paper_ids[i] for i in I[0]]
    return results



In [46]:
# Example usage
folder_path = '/kaggle/input/test12345678'  # Folder where your research papers (text files) are stored
faiss_index, paper_ids = store_paper_embeddings_in_faiss(folder_path)
# print(faiss_index)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/28 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [47]:
# Query the index
query = "What is the impact of AI on healthcare?"
top_papers = query_faiss_index(query, faiss_index, paper_ids)
# print("Top relevant papers:", top_papers)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
len(pinecone_chunks)

67783

In [49]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_bYpHQ_MYJaBXuyz9jvAKrVDCJ9GDWQAS2cPFufcQmgJN8UE6oVzYrMYg3tp4cJ1RV4nVb")
index = pc.Index("research-paper-index")

In [50]:
def batch_upsert(index, vectors, batch_size=1000):
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i:i+batch_size]
        index.upsert(vectors=batch)

In [51]:
batch_upsert(index, pinecone_chunks)
# index.upsert(vectors=pinecone_chunks)

In [ ]:
# 357494

In [78]:
query_embedding = model.encode(["What are the main differences between small data and big data in terms of volume, variety, velocity, and their respective goals?"], convert_to_numpy=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [81]:

query_embedding_2 = [float(i) for i in query_embedding[0]]
# query_embedding_2

In [80]:
response = index.query(
    vector= query_embedding_2,
    top_k=10,
    # include_values=True,
    include_metadata=True,
)
print(response)

# AttributeError: 'numpy.float32' object has no attribute '_composed_schemas'


{'matches': [{'id': 'paper_591#chunk_22',
              'metadata': {'chunk': 'describe the characteristics of big data '
                                    '[54], [55] in contrast\n'
                                    'with the small data [56]:\n'
                                    '• Volume – sheer quantity of the data, '
                                    'small data could have\n'
                                    'limited to large volume, where big data '
                                    'always have an\n'
                                    'enormous volume.\n'
                                    '• Variety – small data are are '
                                    'constrained in terms of their\n'
                                    'breadth and diversity, while big data '
                                    'draw insights from\n'
                                    'a wide spectrum range of data types.\n'
                                    '• Velocity – small data are 